In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime

pd.set_option('display.max_columns',50)

In [2]:


class deck_poker:
    def __init__(self, name='poker', player_cards=[]):
        self.name = name
        self.deck_cards = []
        
        a={}
        for k in range(1,11):a[k]=[]
        self.player_cards = a
        
        self.table_cards = []
        
        
        
    def create_deck(self, range_cards=list(range(2,15)), colors=['p','f','q','c']):
        def list_word(l, word):
            l2=[]
            for li in l:
                l2.append(str(li)+word)
            return l2 
        t = []
        for color in colors:
            k = list_word(range_cards, color)
            t =t + k
        self.deck_cards = t
        return self
    
    def shuffle(self):
        import random
        self.deck_cards = random.sample(self.deck_cards, len(self.deck_cards))
        return self
    
    def take(self, n, player=1):
        k= self.deck_cards[:n]
        self.deck_cards= self.deck_cards[n:]
        if player ==0:
            self.table_cards.extend(k)
        if player >=1:
            self.player_cards [player] = k

        return self
    


In [3]:
def scala_colore(values, colors)  :  
    k=list(map(lambda x:x,Counter(colors).most_common()))[0]
    if k[1]>=5:
        values=np.array(values)[list(np.where(np.array(colors[0:10]) == k[0])[0])]
        s=scala(values)
        if s[1]:
            return 'scala_colore', True, s[2]+1000
        else:
            return 'no', False,0
    else:
        return 'no', False,0

def colore(colors,values):
    somma=0
    k=list(map(lambda x:x,Counter(colors).most_common()))[0]
    if k[1]>=5:
        somma=sum(np.array(values)[list(np.where(np.array(colors) == k[0])[0][0:5])])
        return 'colore', True, somma+500
    else:
        return 'NO', False, 0



def scala(values):
    import numpy as np
    k = np.diff(np.sort(np.array(list(values)))[::-1])
    if len(k[0:len(k)-2][k[0:len(k)-2]==-1])==4:
        return 'scala', True, sum(values[0:(len(values)-2)])+200
    elif len(k[1:len(k)-1][k[1:len(k)-1]==-1])==4:
        return 'scala', True, sum(values[1:(len(values)-1)])+200
    elif len(k[2:len(k)][k[2:len(k)]==-1])==4:
        return 'scala', True, sum(values[2:len(values)])+200
    else: 
        return 'no', False


def poker_tris(values, val):
    t=list(map(lambda x:x,Counter(values).most_common()))[0]

    if t[1]>=val:
        if val==4:
            point =t[0]*4+750
            res ='poker'
        elif val ==3:
            point = t[0]*3+120
            res ='tris'
        return res, True, point
    else:
        return 'no', False,0

def full(values):
    t=list(map(lambda x:x,Counter(values).most_common()))[0:2]
    if t[0][1]>=3:
        try:
            if t[1][1]>=2:

                return 'full',True, t[0][0]*3+t[1][0]*2+300
            else:
                return 'no',False,0
        except:
            return 'no', False, 0
    else:
        return 'no',False, 0

def doppia_coppia(values):
    t=list(map(lambda x:x,Counter(values).most_common()))[0:2]
    if t[0][1]>=2:
        try:
            if t[1][1]>=2:
                return 'doppia_coppia',True,t[0][0]*2+t[1][0]*2+50
            else:
                return 'no',False,0
        except: return 'no', False,0
    else:
        return 'no',False,0

def coppia(values):
    t=list(map(lambda x:x,Counter(values).most_common()))[0]
    if t[1]>=2:
        return 'coppia',True, t[0]*2+20
    else:
        return 'no',False, 0

def high_card(values):
    cards=[]
    for card in values:
        cards.append(int(card[:-1]))

    k = np.array(list(set(cards)))
    k = np.sort(k)[::-1][0]
    return 'high_card',True,k


def point(values,colors,hand_cards):
    carta_alta =high_card(hand_cards)[2]
    A=scala_colore(values,colors)
    if A[1]:return A[0],A[2]
    A=poker_tris(values,4)
    if A[1]:return A[0],A[2]
    A=colore(colors,values)
    if A[1]:return A[0],A[2]
    A=full(values)
    if A[1]:return A[0],A[2]
    A=scala(values)
    if A[1]:return A[0],A[2]
    A=poker_tris(values,3)
    if A[1]:return A[0],A[2]
    A=doppia_coppia(values)
    if A[1]:return A[0],A[2]
    A=coppia(values)
    if A[1]:return A[0],A[2]
    A=high_card(hand_cards)
    return A[0],A[2]






        
            

In [4]:
class game:
    def __init__(self, name='partita_poker', blind=0.005):
        self.name = name
        self.deck = deck_poker()
        self.players=0
        self.turni=[]
        self.inizio=[]
        self.bet = True
        self.piatto=0
        self.max_bet=0
        self.scommesse=1
        self.blind=blind
        
        a={}
        for k in range(1,11):a[k]=('no', 0)
        self.punti = a
        
        a={}
        for k in range(1,11):a[k]='0.000'
        self.action = a
        
        a={}
        for k in range(1,11):a[k]=0
        self.capitale = a
        
        a={}
        for k in range(1,11):a[k]=0
        self.investimento = a
        
        
    
    def start(self):
        dt = datetime.now()
        rand=int(dt.microsecond**2/(dt.year+dt.month*dt.second+dt.day*dt.minute+dt.hour+1))
        np.random.seed(rand)
        
        self.players = int(round(np.random.uniform(low=2, high=10, size=None),0))
        inizio =  int(round(np.random.uniform(low=1, high=self.players, size=None),0))
        k=[]
        k2=[]
        for i in range(1,self.players+1):
            if i>=inizio: k.append(i)
            else: k2.append(i)
        k.extend(k2)
        self.turni = k
        
        
        self.inizio=k[2:]
        k2=k[0:2]
        self.inizio.extend(k2)
        
        
        # mescolo mazzo
        self.deck =deck_poker()
        self.deck=self.deck.create_deck().shuffle()
        
        # creo casualmente il capitale
        for k in range(1,self.players+1):
            self.capitale[k]=round(np.random.uniform(low=0, high=1, size=None),3)
        
        
        #resetto carte
        self.deck.table_cards = []
        a={}
        for k in range(1,11):a[k]=[]
        self.deck.player_cards = a
        
        
        #resetto action
        
        self.bet=False
        a={}
        for k in range(1,11):a[k]= '0.000'
        self.action = a
        
        a={}
        for k in range(1,11):a[k]= 0
        self.investimento = a
        
        self.scommesse=1
        
        self.bet==True
        
        return self
    
    def simulate(self):
        # distribuisco le carte
        for i in self.turni:
            self.deck.take(n=2, player=i)
        # prendo le carte dal deck
        self.deck.take(n=5, player=0)
        for i in self.turni:
            
            self.deck.player_cards[i].extend(self.deck.table_cards)
            hand_cards=self.deck.player_cards[i][0:2]
            values=[]
            colors=[]
            for gi in np.array(self.deck.player_cards[i]):
                for color in ['p','f','q','c']:
                    if color in gi:
                        values.append(int(gi.replace(color,'')))
                        colors.append(color)

            a=list(zip(values,colors))

            def getKey(item):
                 return item[0]

            s=sorted(a, key=getKey, reverse= True)

            values =[]
            colors=[]
            for si in s:
                values.append(si[0])
                colors.append(si[1])
            
            self.punti[i]=point(values, colors,hand_cards)
        
        
        return self
        
    

In [9]:
class play_poker:
        def __init__(self, name='partita_poker'):
            self.name = name
            self.play= game()
            self.play.deck = deck_poker()
            
        def calculate_probability(self,n_sim=0, salva=20000, name_output='poker_dataset.csv'):
            
            df2=pd.DataFrame(list(zip([],[],[],[],[])), index=range(1,11), columns=['punti', 'comb_migliore' ,'CardA',  'CardB',  'giocatori','classifica'])

            num=0
            num_comp=0
            while True:
                if (num_comp >= n_sim) and (n_sim!=0):
                    return df2
                    break

                else:
                    self.play = game()
                    self.play.start()
                    self.play.simulate()
                    num+=1


                    n_players=[self.play.players]*10
                    t2=pd.DataFrame(n_players,columns=['giocatori'])

                    punti_numerici=[]
                    cosa=[]
                    index=[range(1,11)]
                    carta1=[]
                    carta2=[]
                    carta3=[]
                    carta4=[]
                    for k in range(1,11):
                        punti_numerici.append(self.play.punti[k][1])
                        cosa.append(self.play.punti[k][0])
                        j=self.play.deck.player_cards[k][0:2]
                        t=[]
                        for ji in j:
                            t.append(int(ji[:-1]))    
                        t.sort(reverse=True)

                        try:
                            a=t[0]
                            if a==14:
                                a=1
                            carta1.append(a)
                            carta3.append(t[0])
                        except:
                            carta1.append(np.nan)
                            carta3.append(np.nan)
                        try:
                            a=t[1]
                            if a==14:
                                a=1
                            carta2.append(a)
                            carta4.append(t[1])
                        except:
                            carta2.append(np.nan)
                            carta4.append(np.nan)


                    df=pd.DataFrame(list(zip(punti_numerici,cosa,carta1,carta2,carta3,carta4)), index=range(1,11), columns=['punti', 'comb_migliore','CardA','CardB','c','d'])

                    df=df.sort_values(by=['punti','c','d'],ascending=False)
                    g=pd.DataFrame([1,0,0,0,0,0,0,0,0,0][0:self.play.players],columns=['classifica'])
                    df=pd.concat([df.reset_index(drop=True),t2,g], axis=1).dropna().drop(['c','d'],axis=1)
                    
                    df2=pd.concat([df2,df],axis=0).dropna().reset_index(drop=True)
                    if (num % salva) == 0:
                        print('saved')
                        num_comp=num_comp+num
                        num=0

                        df2.to_csv(name_output, index=False)




In [ ]:
p=play_poker()
p.calculate_probability(n_sim=0,salva=50000, name_output='poker_dataset3.csv')

In [ ]:
deck = deck_poker()
deck.create_deck().shuffle()
play= game()
play.start()
def salva_df(play,turn,phase,giro):    
    n_player=[play.players,play.players,play.players,play.players,play.players,play.players,play.players,play.players,play.players,play.players]
    max_bet=[play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet,play.max_bet]
    piatto=[play.piatto,play.piatto,play.piatto,play.piatto,play.piatto,play.piatto,play.piatto,play.piatto,play.piatto,play.piatto]

    punti_numerici=[]
    cosa=[]
    index=[range(1,11)]
    carta1=[]
    carta2=[]
    for k in range(1,11):
        punti_numerici.append(play.punti[k][1])
        cosa.append(play.punti[k][0])
        try:
            a=play.deck.player_cards[k][0]
            if a[:-1]=='14':
                a=a.replace('14','1')
            carta1.append(a)
        except:
            carta1.append(np.nan)
        try:
            a=play.deck.player_cards[k][1]
            if a[:-1]=='14':
                a=a.replace('14','1')
            carta2.append(a)
        except:
            carta2.append(np.nan)


    giocatore=[1,2,3,4,5,6,7,8,9,10]
    fase=[phase]*10
    turno=[str(turn)]*10

    table_card=[]

    for k in range(0,5):
        try:
            table_card.append(play.deck.table_cards[k])
        except:
            table_card.append('NULL')
    table_card1=[table_card[0]]*10
    table_card2=[table_card[1]]*10
    table_card3=[table_card[2]]*10
    table_card4=[table_card[3]]*10
    table_card5=[table_card[4]]*10


    cap=[]
    for k in range(1,11):
        cap.append(play.capitale[k])
    act=[]
    for k in range(1,11):
        act.append(play.action[k][:-6])
    val_act=[]
    for k in range(1,11):
        val_act.append(play.action[k][-5:])

    inv=[]
    for k in range(1,11):
        inv.append(play.investimento[k])

    tu=1
    ju=[0,0,0,0,0,0,0,0,0,0,0]
    for k in range(0,len(giro)):
        ju[giro[k]]=tu
        tu+=1
    ju=ju[1:]
    
    tur_max=[0,0,0,0,0,0,0,0,0,0]
    df=pd.DataFrame(list(zip(fase,turno,giocatore,n_player,carta1,carta2,table_card1,table_card2,table_card3,table_card4,table_card5,cap,inv,max_bet,piatto,act,val_act,ju,tur_max)), index=range(1,11), columns=['Fase','phase','giocatore','#_giocatori','Carta1','Carta2','Banco1','Banco2','Banco3','Banco4','Banco5','capitale','inv','max_bet','piatto','azione','val_act','turno','max_turno']).dropna()
    return df

def fold_check() :   
    h=0
    for ply2 in play.turni:
        if play.action[ply2][0:4]=='FOLD':
            h+=1
    if len(play.turni) - h<=1:
        return True
    else:
        return False

def check_finish():    
    h=0
    for ply2 in play.turni:
        if (play.action[ply2][0:4]=='FOLD') or (play.action[ply2][0:6]=='ALL_IN') or (round(play.investimento[ply2],3)==round(play.max_bet,3)):
            h+=1
    if len(play.turni) - h==0:
        return True
    else:
        return False

df=pd.DataFrame(list(zip([],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[])), index=range(1,11), columns=['Fase','phase','giocatore','#_giocatori','Carta1','Carta2','Banco1','Banco2','Banco3','Banco4','Banco5','capitale','inv','max_bet','piatto','azione','val_act','turno','max_turno']).dropna()
# dai le carte ai giocatori
for ply in play.turni:
    play.deck.take(2, player=ply)

# metti il bind
play.capitale[play.turni[0]]=play.capitale[play.turni[0]]-play.blind
play.investimento[play.turni[0]]=play.investimento[play.turni[0]]+play.blind
play.action[play.turni[0]]='CALL: 0.005'
play.capitale[play.turni[1]]=play.capitale[play.turni[1]]-play.blind*2
play.investimento[play.turni[1]]=play.investimento[play.turni[1]]+play.blind*2
play.action[play.turni[1]]='CALL: 0.01'
# valorizza il piatto e la piu grande scommessa del momento
play.piatto=0.015
play.max_bet=play.blind*2

fasi =['1-PREFLOP', '2-FLOP', '3-TURN', '4-RIVER']
#fasi =['PREFLOP']
for fase in fasi:
    all_fold=0
    ki=0
    play.scommesse=1
    if fase != '1-PREFLOP':
        play.bet=True
        giro = play.turni
    else:
        play.bet=False
        giro= play.inizio
        moltiplicatore=1
    if fase =='2-FLOP':
        ti=3
        play.deck.take(4,0)
        play.deck.table_cards = play.deck.table_cards[1:4]
        moltiplicatore=1
    elif fase=='3-TURN':
        play.deck.take(2,0)
        play.deck.table_cards.pop(3)
        moltiplicatore=2
    elif fase=='4-RIVER':
        play.deck.take(2,0)
        play.deck.table_cards.pop(4)
        moltiplicatore=2

    while True: # gira infinito finchè o tutti foldano salvo uno o se hanno effettuato call
        ki+=1
        if (all_fold==1) or (check_finish()==True and ki>1) :
            df['max_turno'] = int(df['phase'].max())
            break

        for ply in giro: # giriamo per ogni giocatore, da quelli che non hanno blindato
            if (fold_check())==True and play.action[ply][0:4] != 'FOLD':
                play.action[ply] = 'LAST: '+str(play.action[ply])
                all_fold=1
            # se il giocatore è l'unico che non ha foldato, finisce la partita

            elif (play.action[ply][0:4] != 'FOLD') and (play.action[ply][0:6] != 'ALL_IN') : # se non ha foldato o ha fatto all_in
                if play.bet==False: # se non hanno ancora fatto una scommessa
                    choice=['check','bet']
                    random=int(np.random.uniform(low=0, high=1.99999999999999, size=None))
                    if (choice[random]=='bet') & (play.capitale[ply]>= play.max_bet+0.01*moltiplicatore) :
                        # se scommette, si vede se il capitale e piu alto almeno della scommessa minima

                        play.max_bet = round(play.max_bet+0.01*moltiplicatore,3)
                        play.scommesse+=1
                        play.piatto=play.piatto+play.max_bet-float(play.action[ply][-4:])
                        play.capitale[ply]=play.capitale[ply]-play.max_bet
                        play.investimento[ply]=play.investimento[ply]+play.max_bet-float(play.action[ply][-4:])
                        play.action[ply] = 'RAISE: '+str(round(play.max_bet,3))
                        play.bet=True
                    elif (choice[random]=='bet') & (play.capitale[ply]< play.max_bet+0.01*moltiplicatore) & (play.capitale[ply]> play.max_bet+0.001) :
                        # se scommette, si vede se il capitale e piu alto almeno della scommessa minima

                        play.piatto=play.piatto+play.capitale[ply]-float(play.action[ply][-4:])
                        play.max_bet= round(play.max_bet+ play.capitale[ply],3)
                        play.investimento[ply]=play.investimento[ply]+play.capitale[ply]-round(float(play.action[ply][-4:]),3)
                        play.scommesse+=1
                        play.action[ply] = 'ALL_IN: '+str(float(round(play.capitale[ply],3)))
                        play.capitale[ply]=0
                        play.bet=True

                    else: # check
                        play.action[ply] = 'CHECK: 0.00'

                elif (play.bet==True)  & (float(play.action[ply][-4:])<=play.max_bet): # se hanno già fatto una scommessa e scommesso meno del massimo
                    choice=['call','fold','raise']   #se hanno fatto meno di 4 raise, possono scegliere anche raise
                    if play.scommesse<4:
                        random=int(np.random.uniform(low=0, high=3, size=None))
                    else:
                        random=int(np.random.uniform(low=0, high=2, size=None))
                    if (choice[random]=='call') :   # risponde alla scommessa
                        if (play.capitale[ply]>= play.max_bet) : # se il capitale può coprire la massima scommessa
                            play.piatto=play.piatto+play.max_bet-float(play.action[ply][-4:])
                            play.capitale[ply]=play.capitale[ply]- play.max_bet+float(play.action[ply][-4:])
                            play.investimento[ply]=play.investimento[ply]+play.max_bet-float(play.action[ply][-4:])
                            play.action[ply] = 'CALL: '+str(round(play.max_bet,3))
                        else:   # se non può, mette tutto quello che ha
                            play.piatto=play.piatto+play.capitale[ply]-float(play.action[ply][-4:])
                            play.investimento[ply]=play.investimento[ply]+play.capitale[ply]-round(float(play.action[ply][-4:]),3)
                            play.action[ply] = 'ALL_IN: '+str(round(float(play.capitale[ply]),3))
                            play.capitale[ply]=0
                    elif choice[random]=='raise':  # fa un raise
                        if (play.capitale[ply]>= play.max_bet+0.01*moltiplicatore): # se può scommettere piu del minimo, lo fa                   
                            play.max_bet = round(play.max_bet+0.01*moltiplicatore,3)
                            play.scommesse+=1
                            play.piatto=play.piatto+play.max_bet-float(play.action[ply][-4:])
                            play.capitale[ply]=play.capitale[ply]-play.max_bet+float(play.action[ply][-4:])
                            play.investimento[ply]=play.investimento[ply]+play.max_bet-float(play.action[ply][-4:])
                            play.action[ply] = 'RAISE: '+str(round(play.max_bet,3))
                        elif (play.capitale[ply]> play.max_bet+0.001) & (play.capitale[ply]< play.max_bet+0.01*moltiplicatore):
                             #altrimenti scommette quello che gli è rimasto
                            play.piatto=play.piatto+play.capitale[ply]-float(play.action[ply][-4:])
                            play.max_bet= round(play.max_bet+ play.capitale[ply],3)
                            play.investimento[ply]=play.investimento[ply]+play.capitale[ply]-round(float(play.action[ply][-4:]),3)
                            play.scommesse+=1
                            play.action[ply] = 'ALL_IN: '+str(float(round(play.capitale[ply],3)))
                            play.capitale[ply]=0
                    else: #fold
                        play.action[ply] = 'FOLD: 0.00' # altrimenti folda
                        pass
                elif (float(play.action[ply][-4:])>=play.max_bet): # se già ja coperto il max_bet, passa il turno
                        play.action[ply] = 'PASS: '+str(play.action[ply])




        #salviamo
        df=df.append(salva_df(play,ki,fase,giro))




punti_numerici=[]
cosa=[]
for i in range(1,play.players+1):
        
        play.deck.player_cards[i].extend(play.deck.table_cards)
        hand_cards=play.deck.player_cards[i][0:2]
        values=[]
        colors=[]
        for gi in np.array(play.deck.player_cards[i]):
            for color in ['p','f','q','c']:
                if color in gi:
                    values.append(int(gi.replace(color,'')))
                    colors.append(color)

        a=list(zip(values,colors))

        def getKey(item):
             return item[0]

        s=sorted(a, key=getKey, reverse= True)

        values =[]
        colors=[]
        for si in s:
            values.append(si[0])
            colors.append(si[1])

        play.punti[i]=point(values, colors,hand_cards)
        punti_numerici.append(play.punti[i][1])
        cosa.append(play.punti[i][0])
        
df['Carta3']=df['Carta1'].astype(str).str[:-1].astype(np.int64)
df['Carta4']=df['Carta2'].astype(str).str[:-1].astype(np.int64)
df['Carta_max']=df[["Carta3", "Carta4"]].max(axis=1)
df['Carta_min']=df[["Carta3", "Carta4"]].min(axis=1)

ind=range(1,play.players+1)
tg=pd.DataFrame(zip(ind, cosa,punti_numerici), columns=['giocatore','best_comb','point'])
df=pd.merge(df, tg, on='giocatore')
dfk=df.sort_values(['point','Carta_max','Carta_min'],ascending=False)
juy=list(dfk[['giocatore','point','Carta_max','Carta_min']]['giocatore'].unique())
df2=pd.DataFrame(list(zip(juy,list(range(1,len(juy)+1)))), columns=['giocatore','classifica'])
                 
df=pd.merge(df, df2, on='giocatore').sort_values(['Fase','phase']).drop(['Carta3','Carta4'],axis=1)

k_fin=df[(df['Fase']=='4-RIVER') & (df['azione']!='FOLD') ].sort_values('classifica').append(df[(df['Fase']=='4-RIVER') & (df['azione']=='FOLD') ].sort_values('classifica'))
k_fin['classifica_reale']=list(range(k_fin.shape[0]+1))[1:]

df=pd.merge(df, k_fin[['giocatore','classifica_reale']], on='giocatore')
df[(df['Fase']=='4-RIVER') & (df['azione']=='FOLD') ].sort_values('classifica')

df.sort_values(['Fase','phase'])